In [2]:
import pandas as pd
import statsmodels as sm

pd.set_option('display.max_rows',10000)
pd.set_option('display.min_rows',2000)
pd.set_option('display.column_space',30)
pd.set_option('display.max_colwidth',150)
pd.set_option('display.expand_frame_repr',True)

_end_dates    = pd.Series(pd.date_range(start="2020-10-08", end="2021-03-31", freq="D", tz='America/New_York'))
_start_dates  = _end_dates - pd.Timedelta(days=7)
_center_dates = _end_dates - pd.Timedelta(days=3.5)
end_dates     = _end_dates.apply(lambda x: x.timestamp())
center_dates  = _center_dates.apply(lambda x: x.timestamp())
start_dates   = _start_dates.apply(lambda x: x.timestamp())
_end_dates    = _end_dates.apply(lambda x:str(x).split(' ')[0])
_center_dates = _center_dates.apply(lambda x:str(x).split(' ')[0])
_start_dates  = _start_dates.apply(lambda x:str(x).split(' ')[0])

id_l = '/home/casonk/path/mmani_root/mmani0/shared_data/hot/push_file/IDL/'

j=1
k=21
r=5

In [3]:
guns_tl = []
guns_ei = []
guns_dt = []
n=5
# n=10
for date in _center_dates:
    ei_df = pd.read_pickle((id_l + date + ('/EI_DF_{}_{}_{}.pkl').format(j, k, r)))
    top_50 = ei_df.sort_values(by=['Total_Links'], ascending=False).head(50)
    comm_sub_pairs = pd.read_pickle((id_l + date + ('/CSP_{}_{}_{}.pkl').format(j, k, r)))
    top_ns_50 = comm_sub_pairs.loc[top_50.index].groupby(level=0, axis=0).apply(lambda x : x.head(n)).droplevel(0)
    coms = top_ns_50.index.get_level_values(0)
    subs = top_ns_50.index.get_level_values(1)
    dekka = pd.DataFrame({'com':coms, 'sub':subs})
    insubs = ['guns']
    match = dekka[dekka['sub'].isin(insubs)]
    if len(match['com'].unique()) == 1:
        incom = int(match['com'].unique()[0])
        guns_tl += [ei_df.loc[incom, ['Total_Links','EI_Index']][0]]
        guns_ei += [ei_df.loc[incom, ['Total_Links','EI_Index']][1]]
        guns_dt += [date]
        print(date, incom)
    else:
        print('**', date, len(match['com'].unique()), 'comm')

2020-10-04 494
2020-10-05 413
2020-10-06 417
2020-10-07 281
2020-10-08 277
2020-10-09 7452
2020-10-10 65
2020-10-11 290
2020-10-12 1658
2020-10-13 367
2020-10-14 11632
2020-10-15 199
2020-10-16 81
2020-10-17 246
2020-10-18 364
2020-10-19 799
2020-10-20 515
2020-10-21 503
2020-10-22 337
2020-10-23 10456
2020-10-24 10846
2020-10-25 373
2020-10-26 516
2020-10-27 254
2020-10-28 4174
2020-10-29 259
2020-10-30 247
2020-10-31 213
2020-11-01 454
2020-11-02 10502
2020-11-03 3259
2020-11-04 1
2020-11-05 457
2020-11-06 291
2020-11-07 407
2020-11-08 12421
2020-11-09 209
2020-11-10 92
2020-11-11 1118
2020-11-12 99
2020-11-13 258
2020-11-14 488
2020-11-15 332
2020-11-16 154
2020-11-17 10965
2020-11-18 187
2020-11-19 8898
2020-11-20 230
2020-11-21 326
2020-11-22 12828
2020-11-23 12592
2020-11-24 10955
2020-11-25 45
2020-11-26 13974
2020-11-27 5116
2020-11-28 333
2020-11-29 3049
2020-11-30 25
2020-12-01 158
2020-12-02 260
2020-12-03 555
2020-12-04 502
2020-12-05 341
2020-12-06 802
2020-12-07 1721
2020

In [ ]:
from statsmodels.formula.api import ols

In [9]:
df = pd.DataFrame({'x': range(len(guns_dt)), 'y': guns_ei})
lm = ols("y ~ x", df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     232.0
Date:                Sun, 28 Aug 2022   Prob (F-statistic):           8.88e-34
Time:                        09:48:05   Log-Likelihood:                 436.86
No. Observations:                 175   AIC:                            -869.7
Df Residuals:                     173   BIC:                            -863.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1453      0.003    -48.129      0.0